# Dealing with categorical features

In this notebook we cover examples of common tasks for treating **categorical data** prior to modeling. Categorical data needs a lot of attention during data pre-processing. This is because most machine learning algorithms don't deal directly with categorical data. Instead we need to **recode** the data from categorical into numeric, and we will see how we do that in this notebook.

Let's begin by reading some data. We will use a marketing data set of bank customers. You can read more about the data [here](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing). 

In [ ]:
import pandas as pd
import numpy as np

bank = pd.read_csv('data/bank-full.csv', sep = ";")
bank.head()

We can see that our data contains many categorical columns, including the target itself. Let's check the data types:

In [ ]:
bank.dtypes

We can use the `select_dtypes` method to limit the data to just the categorical columns.

In [ ]:
bank.select_dtypes('object').head()

### Exercise

Write a loop to obtain counts of unique values for each categorical column in the data. We can use the `value_counts` method to get counts for a column. 

### End of exercise

It turns out there are **two kinds of data types for categorical data** in `pandas`: `object` and `category`. By default, any string column will inherit the `object` type, but we can later convert them to `category` type. Ideally, a `catogory` type is only appropriate for a column with **a limited number pre-defined categories**. This is because the `category` type is a more rigid data type that we can use to impose additional structure on the column. So this only makes sense when the categories are known and few. Let's illustrate that by turning some of the columns in our data into a `category` columns.

In [ ]:
cat_cols = ['marital', 'default', 'housing', 'loan']
bank[cat_cols] = bank[cat_cols].astype('category')

Why would we want to add additional rigidity? Because this way we can impose some amount of **data integrity**. For example, if `marital` should always be limited to "single", "divorced" or "married" then by converting `marital` into a `category` column we can prevent the data from introducing any other category without first adding it as one of the acceptable categories for this column.

In [ ]:
bank['marital'].cat.categories

### Exercise

Try to change `marital` at the second row to the value "widowed". You should get an error. To fix the error, you need to add "widowed" as one of the acceptable categories. Use the `cat.add_categories` method to add "widowed" as a category and then try again to make sure you don't get an error this time.

Undo your change by reassigning `marital` at the second row to the value "single". Get a count of unique values for `marital` now. Do you notice anything? Explain what and why?

Categorical columns have other useful methods, and their names speak for themselves, such as
`as_ordered`, `as_unordered`, `remove_categories`, `remove_unused_categories`, `rename_categories`, `reorder_categories`, and `set_categories`. It is important to be aware of this functionality and use it when it makes sense. Of course an alternative to using these is to convert the column back to `object` and make all the changes we want and then turn it back into `category`.

### End of exercise

So we saw that a `category` column has pre-defined categories and a set of methods specific to itself for changing the categories, whereas an `object` column is more a type of **free-form** categorical column where the categories can be changed on the fly and no particular structure is imposed. One way the above distinction matters if when we need to rename the categories for a categorical column. This is sometimes referred to as **recoding** or **remapping**.

### Exercise

Let's first begin with an example using `job`, which has type `object`. Rename the category "management" to "managerial". HINT: find all rows where `job` is the string `'management'`, and use `loc` to change those rows to the string `'managerial'`.

The above approach works fine, but it's tedious if we have a lot of changes we want to make. The better way to do it is to create a Python dictionary that maps old values (values we want to change) to new values, then use the `replace` method to replace them all at once.

Create such a dictionary and use `replace` to make the following changes in the `job` column:

- rename `'student'` to `'in-school'`
- combine `'housemaid'` and `'services'` into a single group called `'catering'`
- change `unknown` to a missing value, i.e. `np.NaN` (without quotes)

Get a count of unique values for `job` to make sure everything worked. Note that `value_counts()` does not provide count for missing values by default. We need to specify `dropna = False` to include missing vaules in the count.

### End of exercise

The `replace` method works equally well with a categorical column of type `category`, however **it changes its type to `object`!** So either we have to convert it back to `category`, or we need to use the `rename_categories` method to replace values, which workes very similar to `replace` in that it accepts a dictionary mapping old values to new ones. Here's an example:

In [ ]:
bank['marital'] = bank['marital'].cat.rename_categories({'married': 'taken'})
bank['marital'].value_counts()

Categorical columns can also be easily generated from numeric columns. For example, let's say we want to have a column called `high_balance` that is `True` when balance exceeds $2,000 and `False` otherwise. Technically this would be a boolean column, but in practice it acts as categorical column. Generating such a column is very easy. We refer to such binary colums as **dummy variables** or **flags** because they single out a group.

In [ ]:
bank['high_balance'] = bank['balance'] > 2000

The process of creating a dummy variable **for each category** of a categorical feature is called **one-hot encoding**. Let's see what happens if we one-hot-encode `marital`.

In [ ]:
bank['marital_taken'] = (bank['marital'] == 'taken').astype('int')
bank['marital_single'] = (bank['marital'] == 'single').astype('int')
bank['marital_divorced'] = (bank['marital'] == 'divorced').astype('int')

In [ ]:
bank.filter(like = 'marital').head()

One-hot encoding is a common enough task that we don't need to do it manually like we did above. Instead we can use `pd.get_dummies` to do it in one go.

In [ ]:
pd.get_dummies(bank['marital'], prefix = 'marital').head()

There's an even more streamlined way to do one-hot encoding, although at first blush it appears less straight-forward, but there is a reason it is set up this way and we will explain that later. Just like normalization, one-hot-encoding is a common pre-processing task and we can turn to the `sklearn` library to do the hard part for us.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

bank_cat = bank.select_dtypes('category').copy() # only select columns that have type 'category'
onehot = OneHotEncoder(sparse = False) # initialize one-hot-encoder
onehot.fit(bank_cat)
col_names = onehot.get_feature_names(bank_cat.columns) # this allows us to properly name columns
bank_onehot =  pd.DataFrame(onehot.transform(bank_cat), columns = col_names)
bank_onehot.head()

So we can see that one-hot encoding created a **binary feature** for **each category of each categorical column** in the data. Although to be more specific, we limited it to columns whose type is `category` and excluded columns whose type is `object`. This is because one-hot encoding can quickly blow up the number of columns in the data if we are not careful and include categorical columns with lots of categories (also called **high-cardinality** categorical columns).

What is the point of doing this? The reason we do this is that most machine learning algorithms do not work **directly** with categorical data, so we need to encode the categorical data which turns it into numeric data. One-hot encoding is just one type of encoding, but it is the most common one.

One last note about the `sklearn` pre-processing transformations we learned about in this notebook: If you look at examples online, you may notice that instead of calling `fit` and `transform` separately, you can call `fit_transform` which combines the two steps into one. This may seem reasonable and saves you one extra line of code, but we discourage it. The following exercise will illustrate why, but the main reason will become clear when we talk about machine learning.

### Exercise

Let's return to the data, and once again fit a one-hot encoder on it. This time we run it on `job` and `education`.

In [ ]:
bank_cat = bank[['job', 'education']].copy()
onehot = OneHotEncoder(sparse = False) # initialize one-hot-encoder
onehot.fit(bank_cat)

We now introduce a change: We replace the value for `job` at the second row with `'data scientist'` (assuming that's a new category). Note that `job` is of type `object`, so we can set it to anything we want.

In [ ]:
bank_cat.loc[1, 'job'] = 'data scientist' # introduce a category unseen when we ran fit
bank_cat.head()

The important point here is that we introduce this additional category **after** we ran `fit` on the one-hot encoder above.

Now let's see what happens if we try to run `transform` on the data to one-hot encode the features. If you run the code below you'll notice that we get an error. What is the error for?

In [ ]:
col_names = onehot.get_feature_names(bank_cat.columns)
bank_onehot =  pd.DataFrame(onehot.transform(bank_cat), columns = col_names)
bank_onehot.head()

Is it a good thing that we got an error? The answer is it depends: 

- If we are okay with letting new categories slip through, we can return to where we initiated `OneHotEncoder` and change the `handle_unknown = 'ignore'` (default value is `'error'`). Make this change and rerun the code. What is the one-hot encoded value for `job` at the row that we changed?
- If you want to make sure that we preserve **data integrity** so that the data we call `transform` on matches the schema of the data we ran `fit` on, then we want errors like this to stop us in our tracks so we have a change to see why the data changed. 

### End of exercise

So we saw that using `fit` and `transform`, we can impose a sort of data integrity at training time and enforce it at transform time, and this is true even if the column is of type `object`. This is very similar to what how a column of type `category` works. In fact, if `job` was of type `category` instead of `object`, then we would not have been able to add a new category on the fly, and we would have caught the error pointed in the above exercise earlier.